In [ ]:
import kagglehub

# Download Dataset

path = kagglehub.dataset_download("ravirajsinh45/real-life-industrial-dataset-of-casting-product")

In [ ]:
# Imports

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

plt.style.use('ggplot')
import os

from keras import Sequential, Input
from keras.src.optimizers import Adam, SGD, AdamW
from keras.src.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.src.callbacks import ModelCheckpoint
from keras.src.saving import load_model
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc

In [ ]:
# Check dataset by retrieving images

train_dir = 'dataset/casting_data/train/'
train_def_dir = train_dir + 'def_front/'
train_ok_dir = train_dir + 'ok_front/'

test_dir = 'dataset/casting_data/test/'
test_def_dir = test_dir + 'def_front/'
test_ok_dir = test_dir + 'ok_front/'

fig, axes = plt.subplots(1, 2, figsize=(8, 4))
sample_def = plt.imread(train_def_dir + os.listdir(train_def_dir)[0])
sample_ok = plt.imread(train_ok_dir + os.listdir(train_ok_dir)[0])
axes[0].imshow(sample_def)
axes[1].imshow(sample_ok)
axes[0].set_title('Casting Sample: Defective', loc='left')
axes[1].set_title('Casting Sample: OK', loc='left')
axes[0].grid(False)
axes[1].grid(False)
plt.show()

In [ ]:
# Check dataset for data imbalance

train_len = [len(next(os.walk(train_ok_dir))[2]), len(next(os.walk(train_def_dir))[2])]
test_len = [len(next(os.walk(test_ok_dir))[2]), len(next(os.walk(test_def_dir))[2])]
ok_pct = (train_len[0] + test_len[0]) / (train_len[0] + test_len[0] + train_len[1] + test_len[1]) * 100
def_pct = (train_len[1] + test_len[1]) / (train_len[0] + test_len[0] + train_len[1] + test_len[1]) * 100

labels = [f'OK ({ok_pct:.2f}%)', f'Defective ({def_pct:.2f}%)']

x = range(len(labels))

plt.figure(figsize=(8, 6))
plt.bar(x, train_len, label='Training Set')
plt.bar(x, test_len, bottom=train_len, label='Test Set')

plt.xticks(x, labels)
plt.ylabel('Number of Files')
plt.title('File Distribution by Label and Dataset')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Rescale images and produce validation set

train_generator = ImageDataGenerator(rescale=1. / 255, validation_split=0.2)
test_generator = ImageDataGenerator(rescale=1. / 255)

In [ ]:
# Specify parameters/arguments for data generation

image_size, batch_size, rand_seed = (300, 300), 64, 0

arg_train = {'target_size': image_size,
             'color_mode': 'grayscale',
             'classes': {'ok_front': 0,
                         'def_front': 1},
             'class_mode': 'binary',
             'batch_size': batch_size,
             'seed': rand_seed}

arg_test = {'target_size': image_size,
            'color_mode': 'grayscale',
            'classes': {'ok_front': 0,
                        'def_front': 1},
            'class_mode': 'binary',
            'batch_size': batch_size,
            'seed': rand_seed,
            'shuffle': False}

In [ ]:
# Generate data by iterating through directories

train_set = train_generator.flow_from_directory(
    directory=train_dir, subset='training', **arg_train)

valid_set = train_generator.flow_from_directory(
    directory=train_dir, subset='validation', **arg_train)

test_set = test_generator.flow_from_directory(
    directory=test_dir, **arg_test)

In [ ]:
# Define model architecture and print summary

cnn_model = Sequential([
    Input(shape=image_size + (1,)),

    Conv2D(32, 3, activation='relu', padding='same', strides=2),
    MaxPooling2D(pool_size=2, strides=2),

    Conv2D(64, 3, activation='relu', padding='same', strides=2),
    MaxPooling2D(pool_size=2, strides=2),

    Flatten(),

    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

cnn_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy'])

cnn_model.summary()

In [ ]:
# Training of the model

n_epochs = 20
cnn_model.fit(
    train_set,
    validation_data=valid_set,
    epochs=n_epochs,
    callbacks=ModelCheckpoint(
        'models/model_test.keras',
        save_best_only=True,
        monitor='val_loss'),
    verbose=1)

In [ ]:
# Plot learning curve

histo_dict = cnn_model.history.history
histo_df = pd.DataFrame(histo_dict, index=range(1, n_epochs + 1))
fig, ax = plt.subplots(figsize=(8, 5))
for m in histo_df.columns:
    ax.plot(histo_df.index, m, data=histo_df)
ax.set_xlabel('Epoch')
ax.set_title('Learning Curve', loc='left', weight='bold')
ax.legend()
plt.show()

In [ ]:
# Load saved model and make predictions on the test set

best_model = load_model('models/model_test.keras')

y_pred_prob = best_model.predict(test_set, verbose=1)
y_pred = (y_pred_prob >= 0.5).reshape(-1, )
y_true = test_set.classes[test_set.index_array]

In [ ]:
# Plot ROC curve

y_test = test_set.classes
y_probs = y_pred_prob

if y_probs.shape[1] == 1:
    y_probs = y_probs.ravel()

fpr, tpr, thresholds = roc_curve(y_test, y_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
# Print the report of the parameters

print(classification_report(y_true, y_pred, digits=4))

In [ ]:
# Plot the confusion matrix

fig, ax = plt.subplots(figsize=(4, 3))
ax = sns.heatmap(confusion_matrix(y_true, y_pred), annot=True,
                 annot_kws={'size': 14, 'weight': 'bold'},
                 fmt='d', cbar=False, cmap='Blues')
ax.set_xticklabels(['OK', 'Defective'])
ax.set_yticklabels(['OK', 'Defective'], va='center')
plt.tick_params(axis='both', labelsize=14, length=0)
plt.ylabel('Actual', size=14, weight='bold')
plt.xlabel('Predicted', size=14, weight='bold')
plt.show()

In [ ]:
# Plot 6 examples of the classified images from the test set

class_map = {0: 'OK', 1: 'Defective'}
images, labels = next(iter(test_set))
images = images.reshape(batch_size, *image_size)

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
fig.suptitle('Predictions on Test Images\n', y=0.95, weight='bold', size=16)

# Loop over the first 6 images
for ax, img, label in zip(axes.flat, images[:6], labels[:6]):
    ax.imshow(img, cmap='gray')
    [[pred_prob]] = best_model.predict(img.reshape(1, *image_size, -1))
    pred_label = class_map[int(pred_prob >= 0.5)]
    true_label = class_map[label]
    prob_class = 100 * pred_prob if pred_label == 'Defective' else 100 * (1 - pred_prob)
    ax.set_title(f'Actual: {true_label}', size=12)
    ax.set_xlabel(f'Predicted: {pred_label} ({prob_class:.2f}%)',
                  color='g' if pred_label == true_label else 'r')
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.show()

In [ ]:
# Plot 6 examples of the misclassified images from the test set

misclassified = np.nonzero(y_pred != y_true)[0]
class_map = {0: 'OK', 1: 'Defective'}

# Use only the first 6 misclassified indices
misclassified = misclassified[:6]
batch_num = misclassified // batch_size
image_num = misclassified % batch_size

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
fig.suptitle('Misclassified Test Images\n', y=0.95, weight='bold', size=16)

for ax, bnum, inum in zip(axes.flat, batch_num, image_num):
    images, labels = test_set[bnum]
    img = images[inum]

    ax.imshow(img.reshape(*image_size), cmap='gray')

    [[pred_prob]] = best_model.predict(img.reshape(1, *image_size, -1))
    pred_label = class_map[int(pred_prob >= 0.5)]
    true_label = class_map[labels[inum]]
    prob_class = 100 * pred_prob if pred_label == 'Defective' else 100 * (1 - pred_prob)

    ax.set_title(f'Actual: {true_label}', size=12)
    ax.set_xlabel(f'Predicted: {pred_label} ({prob_class:.2f}%)',
                  color='g' if pred_label == true_label else 'r')
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.show()

In [ ]:
# Compare the ROC curve of all the models

from sklearn.metrics import auc

model_paths = [
    'models/model1_adam.keras',
    'models/model2_sgd.keras',
    'models/model3_sgd50.keras',
    'models/model4_sgd100.keras',
    'models/model5_adamw.keras',
]
colors = ['blue', 'green', 'red', 'purple', 'orange']

y_true = test_set.classes

plt.figure(figsize=(10, 8))
i = 1

for path in model_paths:
    model = load_model(path)

    y_pred = model.predict(test_set, verbose=0).ravel()

    fpr, tpr, _ = roc_curve(y_true, y_pred)
    new_auc = auc(fpr, tpr)

    plt.plot(fpr, tpr, color=colors[i - 1], label=f'Experiment {i} (AUC = {new_auc:.2f})')
    i += 1

plt.plot([0, 1], [0, 1], 'k--')

plt.title('ROC Curves Comparison')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()